<a href="https://colab.research.google.com/github/Ruveyda/HealthChat/blob/master/healthChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Turkish Health-Chatbot**

In [0]:
# libraries

import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.tree import export_graphviz
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [0]:
train = pd.read_csv('Training.csv')
test = pd.read_csv('Testing.csv')

In [0]:
cols = train.columns
cols = cols[:-1]

In [0]:
x = train[cols]
y = train['tahmin']
y1 = y

In [0]:
reduced_data = train.groupby(train['tahmin']).max()

In [0]:
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.33, random_state=42)
testx = test[cols]
testy= test['tahmin']
testy = le.transform(testy)

In [0]:
clf1  = DecisionTreeClassifier()
clf = clf1.fit(x_train,y_train)

In [0]:
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

In [10]:
#feature_importances
for f in range(10):
    print("%d. feature %d - %s (%f)" % (f + 1, indices[f], features[indices[f]] ,importances[indices[f]]))

1. feature 54 - burun_akması (0.026703)
2. feature 93 - iç_kaşıntı (0.026649)
3. feature 79 - kalça_eklemi_ağrısı (0.026626)
4. feature 104 - iştah_artışı (0.026390)
5. feature 128 - tırnaklarda_iltihap (0.025712)
6. feature 73 - el_ve_ayak_şişkinliği (0.025479)
7. feature 118 - balgamda_kan (0.024874)
8. feature 131 - sarı_kabuk_sızıntı (0.024798)
9. feature 2 - nodal_deri_döküntüleri (0.024735)
10. feature 86 - kararsızlık (0.024486)


In [11]:
print("Lütfen aşağıda verilen hastalık belirtilerine sahipseniz evet, değilseniz hayır yazın.") 

Lütfen aşağıda verilen hastalık belirtilerine sahipseniz evet, değilseniz hayır yazın.


In [12]:
def print_disease(node):
    print("Node : ",node)
    node = node[0]
    print("Length of Node : ",len(node))
    val  = node.nonzero() 
    print("Value : ", val)
    disease = le.inverse_transform(val[0])
    return disease
  
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    #print(tree_)
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    #print("def tree({}):".format(", ".join(feature_names)))
    symptoms_present = []
    
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print(name + " ?")
            ans = input()
            ans = ans.lower()
            if ans == 'evet':
                val = 1
                #print("threshold is : ",threshold)
            else:
                #print("threshold is : ",threshold)
                val = 0
            if  val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            print( "Şu hastalığa sahip olabilirsiniz : " +  present_disease )
            red_cols = reduced_data.columns 
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            print("Mevcut semptomlar : " + str(list(symptoms_present)))
            print("Verilen semptomlar :"  +  str(list(symptoms_given)) )  
            confidence_level = (1.0*len(symptoms_present))/len(symptoms_given)
            print("Doğruluk seviyesi : " + str(confidence_level))

    recurse(0, 1)

tree_to_code(clf,cols)

burun_akması ?
hayır
iç_kaşıntı ?
hayır
kalça_eklemi_ağrısı ?
hayır
iştah_artışı ?
hayır
tırnaklarda_iltihap ?
hayır
el_ve_ayak_şişkinliği ?
evet
Node :  [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0. 82.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.]]
Length of Node :  41
Value :  (array([19]),)
['Şu hastalığa sahip olabilirsiniz : Hipotroidi']
Mevcut semptomlar : ['el_ve_ayak_şişkinliği']
Verilen semptomlar :['yorgunluk', 'kilo_alma', 'soğuk eller ve ayaklar', 'ruh_hali_değişimi', 'uyuşukluk', 'baş_dönmesi', 'yüz_ve_göz_şişmesi', 'troid_genişlemesi', 'tırnak_kırılganlığı', 'el_ve_ayak_şişkinliği', 'depresyon', 'sinirlilik', 'anormal_adet_düzeni']
Doğruluk seviyesi : 0.07692307692307693
